In [1]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
installs = pd.read_csv('data/installsConFeatures.csv', low_memory=False, parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,Unnamed: 0,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,top10apps,top50marcas,top10marcas,top10marcasSinNan,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng
created,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,94738,70,1,4432995619177048534,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
2019-04-18 00:00:01.851,94737,70,1,5904733559638204455,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
2019-04-18 00:00:05.152,111877,65,1,896373747754111825,False,True,6287817205707153877,6,102,2,...,3,6,6,6,2,2,1,0,0,8
2019-04-18 00:00:05.589,76146,27,0,3399210824535017892,False,False,6287817205707153877,0,190,2,...,0,0,0,0,21,14,6,1,1,2
2019-04-18 00:00:06.795,435184,339,0,1541425881979513687,False,False,6287817205707153877,0,190,2,...,0,0,0,0,68,46,10,1,1,6


In [3]:
installs.drop(['Unnamed: 0'],axis = 1,inplace=True)

In [4]:
info1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info1['created'] = info1['created'] - pd.Timestamp('2019-04-21 00:00:00')
info1['created'] = info1['created'].transform(lambda x: x.total_seconds())
info1 = info1.reset_index()
info1 = info1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="right")
info1 = info1.rename(columns={'created_x':'label','created_y':'created'})

label1 = info1['label'].to_frame()
train1 = info1.drop('label', axis=1)

In [5]:
info2 = installs['2019-04-22':'2019-04-24'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info2['created'] = info2['created'] - pd.Timestamp('2019-04-22 00:00:00')
info2['created'] = info2['created'].transform(lambda x: x.total_seconds())
info2 = info2.reset_index()
info2 = info2.merge(installs['2019-04-19':'2019-04-21'].reset_index(),on="ref_hash", how="right")
info2 = info2.rename(columns={'created_x':'label','created_y':'created'})

label2 = info2['label'].to_frame()
train2 = info2.drop('label', axis=1)

In [6]:
info3 = installs['2019-04-23':'2019-04-25'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info3['created'] = info3['created'] - pd.Timestamp('2019-04-23 00:00:00')
info3['created'] = info3['created'].transform(lambda x: x.total_seconds())
info3 = info3.reset_index()
info3 = info3.merge(installs['2019-04-20':'2019-04-22'].reset_index(),on="ref_hash", how="right")
info3 = info3.rename(columns={'created_x':'label','created_y':'created'})

label3 = info3['label'].to_frame()
train3 = info3.drop('label', axis=1)

In [7]:
info4 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info4['created'] = info4['created'] - pd.Timestamp('2019-04-24 00:00:00')
info4['created'] = info4['created'].transform(lambda x: x.total_seconds())
info4 = info4.reset_index()
info4 = info4.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
info4 = info4.rename(columns={'created_x':'label','created_y':'created'})

label4 = info4['label'].to_frame()
train4 = info4.drop('label', axis=1)

In [79]:
ventana_ultima = installs['2019-04-24':'2019-04-26'].reset_index()

In [9]:
ventana_ultima

,created,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,top10apps,top50marcas,top10marcas,top10marcasSinNan,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng
0,2019-04-24 00:00:00.178,21,1,2105892908739159676,False,False,6287817205707153877,0,74,0,...,5,0,0,0,0,0,0,1,1,0
1,2019-04-24 00:00:02.223,226,1,208059911196607757,False,True,6287817205707153877,6,163,2,...,10,6,6,6,2,2,1,84,10,8
2,2019-04-24 00:00:02.352,70,1,9071467689792743380,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
3,2019-04-24 00:00:02.353,70,1,9071467689792743380,False,True,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,28,6,9
4,2019-04-24 00:00:02.764,70,1,7588153492836870321,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
5,2019-04-24 00:00:04.010,36,1,8124402120818138602,False,False,6287817205707153877,1,227,2,...,2,1,1,1,2,2,1,1,1,8
6,2019-04-24 00:00:05.468,77,1,3632067678141958815,False,False,6287817205707153877,0,94,0,...,0,0,0,0,0,0,0,1,1,0
7,2019-04-24 00:00:06.381,31,1,4738903240456897851,False,False,6287817205707153877,0,101,0,...,0,0,0,0,0,0,0,1,1,0
8,2019-04-24 00:00:08.243,185,0,2015083770181669009,False,True,6287817205707153877,0,190,2,...,0,0,0,0,35,24,0,0,0,3
9,2019-04-24 00:00:09.630,226,1,5460606787430788722,False,True,6287817205707153877,6,326,2,...,10,6,6,6,2,2,1,0,0,8


In [80]:
train1 = train1.dropna()
train2 = train2.dropna()
train3 = train3.dropna()
train4 = train4.dropna()
ventana_ultima = ventana_ultima.dropna()
label1 = label1.fillna(3*24*60*60)
label2 = label2.fillna(3*24*60*60)
label3 = label3.fillna(3*24*60*60)
label4 = label4.fillna(3*24*60*60)


In [81]:
train1['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-18 00:00:00')).transform(lambda x: x.total_seconds())
train2['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-19 00:00:00')).transform(lambda x: x.total_seconds())
train3['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-20 00:00:00')).transform(lambda x: x.total_seconds())
train4['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-21 00:00:00')).transform(lambda x: x.total_seconds())
ventana_ultima['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-24 00:00:00')).transform(lambda x: x.total_seconds())


In [82]:
train1['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train1))
train2['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train2))
train3['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train3))
train4['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train4))
ventana_ultima['accionesDesdeQueComenzoLaVentana'] = np.arange(len(ventana_ultima))

In [83]:
train1.drop(['created'],axis =1, inplace = True )
train2.drop(['created'],axis =1, inplace = True )
train3.drop(['created'],axis =1, inplace = True )
train4.drop(['created'],axis =1, inplace = True )
ventana_ultima.drop(['created'],axis =1, inplace = True )

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(train4, label4, test_size=0.2, random_state=123)

In [53]:
reg=linear_model.LinearRegression()

In [54]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [55]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((128493, 31), (32124, 31), (128493, 1), (32124, 1))

In [56]:
pred=reg.predict(X_test)

In [57]:
# Square mean square error
mean_squared_error(y_test,pred)**0.5
# np.mean((pred-y_test)**2)

25425.472774204783

In [23]:
######### Ahora para las ventanas que queremos.

In [58]:
ventana_ultima.shape

(160033, 31)

In [59]:
predicciones=reg.predict(ventana_ultima)

In [70]:
ventana_ultima['obj']=predicciones

In [73]:
ventana_ultima['obj'].max()

22409694565776.0

In [74]:
#### Estas predicciones son muy grandes

In [76]:
clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)


In [77]:
clf.partial_fit(train1,label1)
clf.partial_fit(train2,label2)
clf.partial_fit(train3,label3)
clf.partial_fit(train4,label4)


/home/barbara/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/barbara/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/barbara/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/barbara/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [85]:
predicciones2=clf.predict(ventana_ultima)

In [ ]:
#Estas son peores.

In [86]:
predicciones2

array([1.58282998e+45, 2.54902711e+45, 2.65875643e+45, ...,
       2.88342261e+45, 1.04607682e+45, 3.14422995e+45])